In [4]:
import snap7
import ctypes
from snap7.snap7types import *
from snap7.common import check_error
from snap7 import util

In [5]:
plc = snap7.client.Client()
plc.connect('127.0.0.1',0,0,8088)

In [6]:
data_items = (S7DataItem*2)()

In [7]:
data_items[0].Area = ctypes.c_int32(S7AreaDB)
data_items[0].WordLen = ctypes.c_int32(S7WLBit)
data_items[0].Result = ctypes.c_int32(0)
data_items[0].DBNumber = ctypes.c_int32(0)
data_items[0].Start = ctypes.c_int32(8 * 1 + 0) # al estar en modo bit es 8 bits * 5 bytes ya que en el DB0.DBX5.0
data_items[0].Amount = ctypes.c_int32(S7WLBit)  # re

data_items[1].Area = ctypes.c_int32(S7AreaDB)
data_items[1].WordLen = ctypes.c_int32(S7WLBit)
data_items[1].Result = ctypes.c_int32(0)
data_items[1].DBNumber = ctypes.c_int32(0)
data_items[1].Start = ctypes.c_int32(8 * 1 + 1) # + bit
data_items[1].Amount = ctypes.c_int32(S7WLBit)  # re

# data_items[2].Area = ctypes.c_int32(S7AreaDB)
# data_items[2].WordLen = ctypes.c_int32(S7WLReal)
# data_items[2].Result = ctypes.c_int32(0)
# data_items[2].DBNumber = ctypes.c_int32(0)
# data_items[2].Start = ctypes.c_int32(0)
# data_items[2].Amount = ctypes.c_int32(S7WLByte)  # re

# data_items[3].Area = ctypes.c_int32(S7AreaDB)
# data_items[3].WordLen = ctypes.c_int32(S7WLByte)
# data_items[3].Result = ctypes.c_int32(0)
# data_items[3].DBNumber = ctypes.c_int32(0)
# data_items[3].Start = ctypes.c_int32(6)
# data_items[3].Amount = ctypes.c_int32(S7WLByte)  # re

In [8]:
for di in data_items:
    # create the buffer
    buffer = ctypes.create_string_buffer(di.Amount)

    # cast the pointer to the buffer to the required type
    pBuffer = ctypes.cast(ctypes.pointer(buffer),
                          ctypes.POINTER(ctypes.c_uint8))
    di.pData = pBuffer

result, data_items = plc.read_multi_vars(data_items)
for di in data_items:
    check_error(di.Result)



In [9]:
byte_to_value = [util.get_bool, util.get_bool]
result_values = []

# unpack and test the result of each read
for i in range(0, len(data_items)):
    di = data_items[i]
    if di.WordLen == S7WLBit:
        value = util.get_bool(di.pData, 0, 0)
    elif di.WordLen == S7WLByte:
        value = util.get_int(di.pData, 0)
    elif di.WordLen == S7WLDWord:
        value = util.get_dword(di.pData, 0)
    elif di.WordLen == S7WLReal:
        value = util.get_real(di.pData, 0)
    result_values.append(value)
print(result_values)

[True, False]
